# Bathymetry

<https://isde.ie/geonetwork/srv/eng/catalog.search#/metadata/ie.marine.data:dataset.858>

In [1]:
import os
from zipfile import BadZipFile, ZipFile
import matplotlib.pyplot as plt
import contextily as cx
from src import read_data as rd
import rioxarray as rxr

In [2]:
# base data download directory
DATA_DIR = os.path.join("data", "bathymetry")

FILE_NAME = "IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.zip"

URL = (
    "https://gsi.geodata.gov.ie/downloads/Marine/Data/Downloads/"
    "LatestEntireAreaMerge/" + FILE_NAME
)

DATA_FILE = os.path.join(DATA_DIR, FILE_NAME)

# basemap cache directory
cx.set_cache_dir(os.path.join("data", "basemaps"))

In [3]:
rd.download_data(url=URL, data_dir=DATA_DIR, file_name=FILE_NAME)

Data 'IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.zip' already exists in 'data/bathymetry'.
Data downloaded on: 2024-01-04 05:45:48.436471+00:00
Download URL: https://gsi.geodata.gov.ie/downloads/Marine/Data/Downloads/LatestEntireAreaMerge/IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.zip
SHA256 hash: 54e31efe2261b32b5aba377e72170fc439bb6f75042792697b3dd958d7d1ce8c



In [5]:
ZipFile(DATA_FILE).namelist()

['IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.tfw',
 'IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.tif',
 'IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.tif.aux.xml',
 'IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.tif.ovr',
 'IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.tif.xml']

In [5]:
# # extract the archive
# try:
#     z = ZipFile(DATA_FILE)
#     z.extractall(DATA_DIR)
# except BadZipFile:
#     print("There were issues with the file", DATA_FILE)

In [4]:
data = rxr.open_rasterio(DATA_FILE.split(".")[0] + ".tif", chunks="auto")

In [5]:
data

<xarray.DataArray (band: 1, y: 8356, x: 13134)>
dask.array<open_rasterio-260fdaf54627a6b41eb8878a3ae83423<this-array>, shape=(1, 8356, 13134), dtype=float32, chunksize=(1, 5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -25.9 -25.89 -25.89 -25.89 ... -9.09 -9.089 -9.088
  * y            (y) float64 57.44 57.44 57.44 57.44 ... 46.75 46.75 46.75 46.75
    spatial_ref  int64 0
Attributes: (12/15)
    AREA_OR_POINT:           Area
    DataType:                Generic
    RepresentationType:      ATHEMATIC
    STATISTICS_COVARIANCES:  1471471.584787387
    STATISTICS_MAXIMUM:      -16.535715103149
    STATISTICS_MEAN:         -2204.3784584559
    ...                      ...
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       1213.0422848307
    _FillValue:              0.0
    scale_factor:            1.0
    add_offset:              0.0
    units:                   metre

In [6]:
data.rio.crs

CRS.from_epsg(4326)

In [7]:
data.rio.bounds()

(-25.896181999999996, 46.746273090207886, -9.08688467362262, 57.44053899999997)

In [8]:
data.rio.resolution()

(0.00127983076948206, -0.0012798307694820601)

In [9]:
# read Kish Basin extent
_, extent = rd.read_dat_file(dat_path=os.path.join("data", "kish-basin"))

In [14]:
extent.crs

<Projected CRS: EPSG:23029>
Name: ED50 / UTM zone 29N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - between 12°W and 6°W - Faroe Islands - onshore; Spain - mainland onshore; Ireland offshore.
- bounds: (-12.0, 36.13, -6.0, 62.41)
Coordinate Operation:
- name: UTM zone 29N
- method: Transverse Mercator
Datum: European Datum 1950
- Ellipsoid: International 1924
- Prime Meridian: Greenwich

In [12]:
# convert the extent's CRS to match the raster's
extent_ = extent.to_crs(data.rio.crs)

In [17]:
extent_.bounds

,minx,miny,maxx,maxy
0,-6.208005,53.084514,-5.350585,53.546522


In [4]:
# data.rio.clip(extent_)

In [ ]:
# clip to extent
data = rxr.open_rasterio(
    DATA_FILE.split(".")[0] + ".tif", chunks="auto", masked=True
).rio.clip()
# data = rxr.open_rasterio(FILE_PATH, cache=False, masked=True).rio.clip(
#     ie["geometry"], from_disk=True
# )